In [27]:
import pandas as pd
import numpy as np

from future.utils import iteritems
from pyPRMS.prms_helpers import float_to_str

In [28]:
col_names = ['datatype', 'ctl_param', 'default', 'description']
col_types = [np.str for __ in range(0,4)]
# col_types.extend([np.float for __ in range(0,10)])
cols = dict(zip(col_names, col_types))

# Manually generated control file parameters from setup.c
workfile = '/Users/pnorton/Projects/National_Hydrology_Model/documents/PRMS6/prms_control_file_parameters_raw.txt'
df = pd.read_csv(workfile, sep='\t', dtype=cols, encoding='ascii')
df = df.fillna('')
print(df.info())

# Map datatypes to numbers
datatype_to_num = {'integer':'1', 'integer_array':'1', 'real':'2', 'string':'4', 'string_array':'4'}

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 4 columns):
datatype       136 non-null object
ctl_param      136 non-null object
default        136 non-null object
description    136 non-null object
dtypes: object(4)
memory usage: 4.3+ KB
None


In [29]:
df.tail()

,datatype,ctl_param,default,description
131,string_array,nhruOutVar_names,none,List of variable names for which output is wri...
132,string_array,nsegmentOutVar_names,none,
133,string_array,nsubOutVar_names,none,List of variable names for which output is wri...
134,string_array,statVar_element,none,List of identification numbers corresponding t...
135,string_array,statVar_names,none,List of variable names for which output is wri...


### Generate a basic XML file for the control parameters

In [32]:
import xml.dom.minidom as minidom
import xml.etree.ElementTree as xmlET

ctl_xml = xmlET.Element('control')

for index, row in df.iterrows():
    ctl_entry = xmlET.SubElement(ctl_xml, 'control_param')
    ctl_entry.set('name', row['ctl_param'])
    
    if isinstance(row['default'], basestring):
        xmlET.SubElement(ctl_entry, 'default').text = row['default']
    else:
        xmlET.SubElement(ctl_entry, 'default').text = float_to_str(row['default'])
        
    xmlET.SubElement(ctl_entry, 'type').text = datatype_to_num[row['datatype']]
    xmlET.SubElement(ctl_entry, 'numvals').text = "1"
    
    if row['description'] == '':
        xmlET.SubElement(ctl_entry, 'desc').text = 'none'
    else:
        xmlET.SubElement(ctl_entry, 'desc').text = row['description']
    
#     print(row['ctl_param'], row['default'])

### Write XML file

In [33]:
xmlstr = minidom.parseString(xmlET.tostring(ctl_xml)).toprettyxml(indent='    ')
with open('/Users/pnorton/tmp/control.xml', 'w') as ff:
    ff.write(xmlstr)
    
# print(xmlstr)

In [16]:
print(xmlstr)

NameError: name 'xmlstr' is not defined